In [2]:
# Import libraries for data preprocessing
import numpy as np
import pandas as pd

In [3]:
years = [1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020]

In [4]:
# Load population data
df_population = pd.read_csv("Datasets\\Demographics\\2a369fff-08eb-4ed9-adce-92957698ad90_Data.csv", skiprows=range(218,272))

# Drop columns we don't need
print(df_population.columns)
columns_to_drop = ['Series Name', 'Series Code', 'Country Code']
df_population = df_population.drop(columns=columns_to_drop)

# Estimate the population for 2024
df_population['2024'] = df_population['2023 [YR2023]']*1.091
df_population['2024'] = df_population['2024'].apply(lambda x: int(x))
df_population = df_population.drop(columns=['2023 [YR2023]'])

Index(['Series Name', 'Series Code', 'Country Name', 'Country Code',
       '1980 [YR1980]', '1984 [YR1984]', '1988 [YR1988]', '1992 [YR1992]',
       '1996 [YR1996]', '2000 [YR2000]', '2004 [YR2004]', '2008 [YR2008]',
       '2012 [YR2012]', '2016 [YR2016]', '2020 [YR2020]', '2023 [YR2023]'],
      dtype='object')


In [5]:
# Transpose the data to match the format of the other dataset
df_population_transposed = df_population.melt(id_vars=['Country Name'], var_name='Year', value_name='Population')

# Check if there are missing values
df_population_transposed = df_population_transposed.replace('..', np.nan)
df_population_transposed = df_population_transposed.dropna()
print(df_population_transposed.isnull().sum())

# Convert strings to numerical values
df_population_transposed['Year'] = df_population_transposed['Year'].str.extract(r'(\d+)')
df_population_transposed["Year"] = df_population_transposed["Year"].apply(lambda x: int(x))
df_population_transposed["Population"] = df_population_transposed["Population"].apply(lambda x: float(x))

# Sort the data by country and year
df_population_transposed = df_population_transposed.sort_values(by = ['Country Name', 'Year'], ascending=[True, True])

# Rename columns to match the other dataset
df_population_transposed = df_population_transposed.rename(columns={'Country Name': 'Country'})

# Save cleaned file
df_population_transposed.to_csv('Datasets\\Demographics\\Cleaned\\WB_Population.csv', index=False)

Country Name    0
Year            0
Population      0
dtype: int64


In [6]:
# Load GDP data
df_gdp = pd.read_csv("Datasets\\Demographics\\WEO_Data.xls", delimiter="\t", encoding="UTF-16 LE")

# Keep only the columns we need
print(df_gdp.columns)
columns_to_keep = [str(year) for year in years]
columns_to_keep.insert(0,'Country')
columns_to_keep.append("2024")
df_gdp = df_gdp[columns_to_keep]

# Fill in missing values
df_gdp = df_gdp.fillna(value=0, axis=1)

# Drop rows with missing values
df_gdp = df_gdp.replace('--', np.nan)
df_gdp = df_gdp.dropna()


Index(['Country', 'Subject Descriptor', 'Units', 'Scale',
       'Country/Series-specific Notes', '1980', '1981', '1982', '1983', '1984',
       '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993',
       '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002',
       '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', '2022', '2023', '2024', 'Estimates Start After', 'Unnamed: 51'],
      dtype='object')


In [7]:
# Transpose the data to match the format of the other dataset
df_gdp_transposed = df_gdp.melt(id_vars='Country', var_name='Year', value_name='GDP_share')

# Convert strings to numerical values
df_gdp_transposed["Year"] = df_gdp_transposed["Year"].apply(lambda x: int(x))
df_gdp_transposed["GDP_share"] = df_gdp_transposed["GDP_share"].apply(lambda x: float(x))

# Check if there are missing values
print(df_gdp_transposed.isnull().sum())

# Sort the data by country and year
df_gdp_transposed = df_gdp_transposed.sort_values(by = ['Country', 'Year'], ascending=[True, True])

# Save cleaned file
df_gdp_transposed.to_csv('Datasets\\Demographics\\Cleaned\\WEO_GDP.csv', index=False)

Country      0
Year         0
GDP_share    0
dtype: int64


In [104]:
# Load medals data
df_medals = pd.read_csv("Datasets\\Historical\\Olympic_Games_Medal_Tally.csv")

# Filter for summer games and years we need
df_medals = df_medals.loc[df_medals["year"].isin(years)]
df_medals = df_medals.loc[df_medals["edition"].str.contains("Summer")]

# Drop columns we don't need
cols_to_drop = ["edition_id", "edition", "country_noc"]
df_medals = df_medals.drop(columns=cols_to_drop)


In [105]:
# Merge gpd and population dataframes
df_merged = pd.merge(df_gdp_transposed, df_population_transposed, on=['Country', 'Year'], how='inner')

# Create new rows for missing years
for country in df_merged.Country.unique():
    for year in years:
        if not ((df_medals['country'] == country) & (df_medals['year'] == year)).any():
            new_row = {'year': year, 'country': country, 'gold': 0, 'silver': 0, 'bronze': 0, 'total': 0}
            df_medals = df_medals._append(new_row, ignore_index=True)

# Sort the data by country and year
df_medals = df_medals.sort_values(by = ['country', 'year'], ascending=[True, True])

# Rename columns to match the other dataset
df_medals = df_medals.rename(columns={'country': 'Country', 'year': 'Year'})

# Join medals and merged dataframes and fill in missing values
df_concat = pd.merge(df_merged, df_medals, on=['Country', 'Year'], how='inner')

In [15]:
import requests
from bs4 import BeautifulSoup

def get_number_of_athletes(country_name):

    # List of years for the Summer Olympics Wikipedia pages
    years = [1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020, 2024]

    # Dictionary to store the number of athletes for each country
    athletes_data = dict.fromkeys(years)

    # Iterate over the years
    for year in years:
        # Construct the URL
        url = f"https://en.wikipedia.org/wiki/{year}_Summer_Olympics"

        # Send a request to the URL
        response = requests.get(url)
    
        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
            return None
        
        # Parse the content of the request with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the table with the participating National Olympic Committees
        if year in [1988, 1992]:
            table = soup.find('table', {'class': 'wikitable sortable'})
        elif year == 1980:
            table = soup.find('table', {'class': 'wikitable collapsible collapsed sortable'})
        elif year == 2012:
            table = soup.find('table', {'class': 'wikitable mw-collapsible mw-collapsed sortable'})
        else:
            table = soup.find('table', {'class': 'wikitable collapsible collapsed sortable'})
        
        if not table:
            print(f"Failed to find the table for {year}.")
            return None
        
        # Extract table rows
        rows = table.find_all('tr')
        
        # Iterate over rows to find the country
        for row in rows[1:]:  # Skip the header row
            cols = row.find_all('td')
            if cols:
                country = cols[1].text.strip()
                if country_name.lower() in country.lower():
                    number_of_athletes = cols[-1].text.strip()
                    athletes_data[year] = number_of_athletes
                    break
    
    # Check if the dictionary is empty
    if all(value == None for value in athletes_data.values()):
        print(f"Failed to find the number of athletes for {country_name}.")
        return None
    # Check if there are missing values and replace with 0
    elif any(value == None for value in athletes_data.values()):
        athletes_data = {year: 0 if value is None else value for year, value in athletes_data.items()}
        print(f"Failed to find the number of athletes for some years for {country_name}.")
    
    # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(athletes_data, orient='index', columns=['Number of Athletes'])
    df = df.reset_index().rename(columns={'index': 'Year'})
    country_series = pd.Series([country_name] * len(df), name='Country')
    df = pd.concat([country_series, df], axis=1)
    
    return df

In [16]:
# Get the number of athletes for each country
countries = df_concat['Country'].unique()
df_athletes_all = None
for country in countries:
    print(f"Getting the number of athletes for {country}.")
    df_athletes = get_number_of_athletes(country)
    if df_athletes is not None:
        if country == countries[0]:
            df_athletes_all = df_athletes
        else:
            df_athletes_all = pd.concat([df_athletes_all, df_athletes])
df_athletes_all.to_csv('Datasets\\Demographics\\Cleaned\\Number_of_Athletes.csv', index=False)

Getting the number of athletes for Afghanistan.
Failed to find the number of athletes for some years for Afghanistan.
Getting the number of athletes for Albania.
Failed to find the number of athletes for some years for Albania.
Getting the number of athletes for Algeria.
Getting the number of athletes for Andorra.
Getting the number of athletes for Angola.
Failed to find the number of athletes for some years for Angola.
Getting the number of athletes for Antigua and Barbuda.
Failed to find the number of athletes for some years for Antigua and Barbuda.
Getting the number of athletes for Argentina.
Failed to find the number of athletes for some years for Argentina.
Getting the number of athletes for Armenia.
Failed to find the number of athletes for some years for Armenia.
Getting the number of athletes for Aruba.
Failed to find the number of athletes for some years for Aruba.
Getting the number of athletes for Australia.
Getting the number of athletes for Austria.
Getting the number of 

In [107]:
# Load the number of athletes data
df_athletes_all = pd.read_csv("Datasets\\Demographics\\Cleaned\\Number_of_Athletes.csv")

# Rename columns for merging
# df_athletes_all.rename(columns={'Country': 'c', 'Year': 'y'}, inplace=True)

# Merge the dataframes
df_concat = df_concat[df_concat['Country'].isin(df_athletes_all['Country'])]
df_full = pd.merge(df_concat, df_athletes_all, on=['Country', 'Year'], how='inner')

# Convert the year to datetime
year_dt = pd.to_datetime(df_full["Year"], format='%Y')
df_full.drop(columns=["Year"], inplace=True)
df_full = df_full.join(year_dt)

# Save the final dataset
df_full.to_csv('Datasets\\final.csv', index=False)